In [ ]:
### peak pool

In [3]:
os.chdir("/Charles/project/ASm6A/METTL3_chip/human/peak/")
wt = "shNC_peaks.narrowPeak"
kd = "kd_peaks.narrowPeak"
result_dir = "/Charles/project/ASm6A/METTL3_chip/human/peak/peak_pool/"
##############

def select_the_lowerP(df, result_file):
    res_arrsy = []
    for a, values in df.iterrows():
        if values['pValue_wt'] > values['pValue_kd']:
            res_arrsy.append([values["chrom_wt"], values["chromStart_wt"], values["chromEnd_wt"]])
        else:
            res_arrsy.append([values["chrom_kd"], values["chromStart_kd"], values["chromEnd_kd"]])
    df_res = pd.DataFrame(res_arrsy)
    df_res.to_csv(result_file, sep="\t", header=False, index=False)

res = os.path.join(result_dir, "overlap.txt")
os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (wt, kd, res))
df = pd.read_table(res, sep="\t", header=None)
df.columns = [
    "chrom_wt", "chromStart_wt", "chromEnd_wt", "name_wt", "score_wt",
    "strand_wt", "signalValue_wt", "pValue_wt", "qValue_wt", "peak_wt",
    "chrom_kd", "chromStart_kd", "chromEnd_kd", "name_kd", "score_kd",
    "strand_kd", "signalValue_kd", "pValue_kd", "qValue_kd", "peak_kd"
]
res = os.path.join(result_dir, "overlap_withLowerP.txt")
select_the_lowerP(df, res)
###
res = os.path.join(result_dir, "only_wt.txt")
os.system("bedtools intersect -a %s -b %s -wa -v > %s" % (wt, kd, res))
res = os.path.join(result_dir, "only_kd.txt")
os.system("bedtools intersect -a %s -b %s -wa -v > %s" % (kd, wt, res))

0

In [7]:
os.chdir(result_dir)

! cat overlap_withLowerP.txt only_wt.txt only_kd.txt | awk '{OFS="\t"}{print $1,$2,$3}' -  | sort -k1,1 -k2,2n - | bedtools merge -i - > final_peaks.bed